In [178]:
import numpy as np
from susy_qm import calculate_Hamiltonian
import pennylane as qml
from pennylane import numpy as pnp
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import pandas as pd

In [22]:
cutoff = 4
potential = 'QHO'

In [23]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues, eigenvectors = np.linalg.eig(H)

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

In [24]:
#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [25]:
params = pnp.array([[
            [2.195201075128017,
            3.106916574276913,
            4.187261286195278],
            [3.046766119144531,
            0.025863111078491485,
            3.146307109203438],
            [0.5004353269790704,
            6.258003111574653,
            2.376347569714709]
        ]],  requires_grad=True)

In [26]:
# Define the PennyLane device
dev = qml.device('default.qubit', wires=num_qubits, shots=None)

# Define the parameterized ansatz
@qml.qnode(dev)
def ansatz(params):
    qml.StronglyEntanglingLayers(params, wires=range(num_qubits), imprimitive=qml.CZ)
    return qml.state()  # Return the statevector

# Initialize random parameters for the ansatz
num_layers = 1
params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
#params = np.random.random(params_shape)

# Get the statevector from the ansatz
ansatz_state = ansatz(params)

# Compute the overlap
overlap = np.vdot(min_eigenvector, ansatz_state)  # Inner product
overlap_squared = np.abs(overlap)**2  # Overlap squared

print("Overlap:", overlap)
print("Overlap squared:", overlap_squared)


Overlap: (0.9218182577341993-0.38681378394123034j)
Overlap squared: 0.9993738037390475


In [184]:
def cost_function(params):

    params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
    ansatz_state = ansatz(params)

    overlap = np.vdot(min_eigenvector, ansatz_state)
    overlap_squared = np.abs(overlap)**2  

    return (1 - overlap_squared)

In [187]:
def cost_function_hellinger(params):

    params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
    ansatz_state = ansatz(params)

    min_eigenvector_prob = np.abs(min_eigenvector)**2
    ansatz_prob = np.abs(ansatz_state)**2

    hellinger_fidelity = np.sum(np.sqrt(min_eigenvector_prob * ansatz_prob))

    return (1 - hellinger_fidelity)


In [188]:
x0 = np.random.random(params_shape).flatten()

res = minimize(
    cost_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':10000}
)


In [189]:
res.fun

np.float64(7.740763585672994e-10)

In [192]:
data = []

for potential in ['QHO', 'AHO', 'DW']:
    for cutoff in [2, 4, 8, 16, 32, 64]:

        #calculate Hamiltonian and expected eigenvalues
        H = calculate_Hamiltonian(cutoff, potential)
        eigenvalues, eigenvectors = np.linalg.eig(H)

        min_index = np.argmin(eigenvalues)
        min_eigenvalue = eigenvalues[min_index]
        min_eigenvector = eigenvectors[:, min_index]

        #create qiskit Hamiltonian Pauli string
        hamiltonian = SparsePauliOp.from_operator(H)
        num_qubits = hamiltonian.num_qubits

        num_layers = 1
        params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
        x0 = np.random.random(params_shape).flatten()

        # Initialize device
        dev = qml.device("default.qubit", wires=num_qubits)

        # Define the parameterized ansatz
        @qml.qnode(dev)
        def ansatz(params):
            qml.StronglyEntanglingLayers(params, wires=range(num_qubits), imprimitive=qml.CZ)
            return qml.state()  # Return the statevector
        
        def cost_function(params):

            params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
            ansatz_state = ansatz(params)

            overlap = np.vdot(min_eigenvector, ansatz_state)
            overlap_squared = np.abs(overlap)**2  

            return (1 - overlap_squared)
        
        def cost_function_hellinger(params):

            params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
            ansatz_state = ansatz(params)

            min_eigenvector_prob = np.abs(min_eigenvector)**2
            ansatz_prob = np.abs(ansatz_state)**2

            hellinger_fidelity = np.sum(np.sqrt(min_eigenvector_prob * ansatz_prob))

            return (1 - hellinger_fidelity)
        

        overlap_res = minimize(
            cost_function,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        hellinger_res = minimize(
            cost_function_hellinger,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        row_data = {'potential': potential, 'cutoff': cutoff, 'overlap': overlap_res.fun, 'hellinger': hellinger_res.fun}

        data.append(row_data)     

In [193]:
pd.DataFrame(data)

,potential,cutoff,overlap,hellinger
0,QHO,2,2.220446e-16,0.000000e+00
1,QHO,4,2.220446e-16,1.110223e-16
2,QHO,8,1.110223e-15,2.220446e-16
3,QHO,16,0.000000e+00,8.881784e-16
4,QHO,32,1.332268e-15,3.330669e-16
5,QHO,64,1.554312e-15,4.440892e-16
6,AHO,2,0.000000e+00,4.440892e-16
7,AHO,4,8.881784e-16,2.220446e-16
8,AHO,8,1.415499e-05,7.077518e-06
9,AHO,16,1.149961e-03,5.470980e-04


## Real Amplitudes

In [146]:
cutoff = 4
potential = 'AHO'

In [147]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues, eigenvectors = np.linalg.eig(H)

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

In [148]:
#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [167]:
# Parameters
repetition = 1

# Initialize device
dev = qml.device("default.qubit", wires=num_qubits)
    
# Define the ansatz circuit
@qml.qnode(dev)
def real_amplitudes(params, repetition, num_qubits):
    param_index = 0
    for _ in range(repetition):

        # Apply RY rotations
        for i in range(num_qubits):
            qml.RY(params[param_index], wires=i)
            param_index += 1

        # Apply entanglement
        for j in reversed(range(1, num_qubits)):  # Reverse linear entanglement
            qml.CNOT(wires=[j - 1, j])

    return qml.state()

# Number of parameters required for this ansatz
n_params = repetition * num_qubits
params = np.random.uniform(0, 2 * np.pi, size=n_params)


In [168]:
drawer = qml.draw(real_amplitudes)
print(drawer(params, repetition, num_qubits))

0: ──RY(3.43)─╭●─┤  State
1: ──RY(0.60)─╰X─┤  State


In [169]:
def cost_function(params):

    ansatz_state = real_amplitudes(params, repetition, num_qubits)

    overlap = np.vdot(min_eigenvector, ansatz_state)
    overlap_squared = np.abs(overlap)**2  

    return (1 - overlap_squared)

In [170]:
def cost_function_hellinger(params):

    ansatz_state = real_amplitudes(params, repetition, num_qubits)

    min_eigenvector_prob = np.abs(min_eigenvector)**2
    ansatz_prob = np.abs(ansatz_state)**2

    hellinger_fidelity = np.sum(np.sqrt(min_eigenvector_prob * ansatz_prob))

    return (1 - hellinger_fidelity)


In [171]:
x0 = np.random.uniform(0, 2 * np.pi, size=n_params)

res = minimize(
    cost_function,
    x0,
    method= "COBYLA",
    options= {'maxiter':10000, 'tol': 1e-8}
)


In [182]:
data = []

repetition = 1

for potential in ['QHO', 'AHO', 'DW']:
    for cutoff in [2, 4, 8, 16, 32, 64]:

        #calculate Hamiltonian and expected eigenvalues
        H = calculate_Hamiltonian(cutoff, potential)
        eigenvalues, eigenvectors = np.linalg.eig(H)

        min_index = np.argmin(eigenvalues)
        min_eigenvalue = eigenvalues[min_index]
        min_eigenvector = eigenvectors[:, min_index]

        #create qiskit Hamiltonian Pauli string
        hamiltonian = SparsePauliOp.from_operator(H)
        num_qubits = hamiltonian.num_qubits
        n_params = repetition * num_qubits

        x0 = np.random.uniform(0, 2 * np.pi, size=n_params)

        # Initialize device
        dev = qml.device("default.qubit", wires=num_qubits)

        # Define the ansatz circuit
        @qml.qnode(dev)
        def real_amplitudes(params, repetition, num_qubits):
            param_index = 0
            for _ in range(repetition):

                # Apply RY rotations
                for i in range(num_qubits):
                    qml.RY(params[param_index], wires=i)
                    param_index += 1

                # Apply entanglement
                for j in reversed(range(1, num_qubits)):  # Reverse linear entanglement
                    qml.CNOT(wires=[j - 1, j])

            return qml.state()

        overlap_res = minimize(
            cost_function,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        hellinger_res = minimize(
            cost_function_hellinger,
            x0,
            method= "COBYLA",
            options= {'maxiter':10000, 'tol': 1e-8}
        )

        row_data = {'potential': potential, 'cutoff': cutoff, 'overlap': overlap_res.fun, 'hellinger': hellinger_res.fun}

        data.append(row_data)     

In [183]:
pd.DataFrame(data)

,potential,cutoff,overlap,hellinger
0,QHO,2,0.000000e+00,0.000000e+00
1,QHO,4,0.000000e+00,0.000000e+00
2,QHO,8,0.000000e+00,2.220446e-16
3,QHO,16,2.220446e-16,0.000000e+00
4,QHO,32,4.440892e-16,1.110223e-16
5,QHO,64,4.440892e-16,0.000000e+00
6,AHO,2,0.000000e+00,0.000000e+00
7,AHO,4,7.132422e-02,3.632175e-02
8,AHO,8,4.695409e-02,2.375930e-02
9,AHO,16,4.840966e-02,2.448323e-02
